In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import pandas as pd
import subprocess
import time

from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.linear_model import RidgeClassifier

from tqdm import tqdm
import glob
import os 

In [2]:
# # combine all csv files 
# all_files = glob.glob("features/*.csv")

# li = []
# print("FEATURES")
# for filename in all_files:
#     df = pd.read_csv(filename, sep=';')
#     speaker, session, emotion, start_time, content = filename.split('/')[1:][0].split('_')
#     df.insert(1, 'speaker', speaker)
#     df.insert(2, 'emotion', emotion)
#     df.insert(3, 'session', session)
    
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)
# frame = frame.drop('name',axis=1)
# # write new df to its own file 
# frame.to_csv('opensmile_features.csv')

In [3]:
df = pd.read_csv('opensmile_features.csv', index_col=0)
# header_list = df.columns

# # store identifying info 
# manip = df.loc[:,'speaker':'frameTime']


In [4]:
# find means of neutral speaker utterances
# neutral_df    = df.loc[df['emotion'] == 'neutral']
# group means by speaker
# neutral_means = neutral_df.groupby('speaker').mean().reset_index('speaker')
# neutral_std   = neutral_df.groupby('speaker').std().reset_index('speaker')

# useful for speaker cross-validation
# speaker_list  = list(neutral_means['speaker'])

# # remove irrelevant columns
# neutral_means = neutral_means.drop(['session', 'frameTime'],axis=1)
# neutral_std   = neutral_std.drop(['session', 'frameTime'],axis=1)

In [5]:
# # normalize by speaker 
# df_values = []

# for i in tqdm(range(len(manip['speaker']))):    
#     sp = df.iloc[0]['speaker']
#     content  = df.loc[:,'pcm_RMSenergy_sma_max'::].iloc[i] - neutral_means.loc[neutral_means['speaker'] == sp].loc[:,'pcm_RMSenergy_sma_max'::]
#     labels   =  manip.iloc[[i]].values
#     new_rows = np.hstack((labels, content.values))
#     df_values.append(new_rows)
    
# new_df = pd.DataFrame(np.vstack(df_values), columns = header_list)
# new_df.to_csv('normalized_opensmile_features.csv')


In [44]:
norm_df = pd.read_csv('normalized_opensmile_features.csv', index_col=0)
list(norm_df.columns.values)


['speaker',
 'emotion',
 'session',
 'frameTime',
 'pcm_RMSenergy_sma_max',
 'pcm_RMSenergy_sma_min',
 'pcm_RMSenergy_sma_range',
 'pcm_RMSenergy_sma_maxPos',
 'pcm_RMSenergy_sma_minPos',
 'pcm_RMSenergy_sma_amean',
 'pcm_RMSenergy_sma_linregc1',
 'pcm_RMSenergy_sma_linregc2',
 'pcm_RMSenergy_sma_linregerrQ',
 'pcm_RMSenergy_sma_stddev',
 'pcm_RMSenergy_sma_skewness',
 'pcm_RMSenergy_sma_kurtosis',
 'pcm_fftMag_mfcc_sma[1]_max',
 'pcm_fftMag_mfcc_sma[1]_min',
 'pcm_fftMag_mfcc_sma[1]_range',
 'pcm_fftMag_mfcc_sma[1]_maxPos',
 'pcm_fftMag_mfcc_sma[1]_minPos',
 'pcm_fftMag_mfcc_sma[1]_amean',
 'pcm_fftMag_mfcc_sma[1]_linregc1',
 'pcm_fftMag_mfcc_sma[1]_linregc2',
 'pcm_fftMag_mfcc_sma[1]_linregerrQ',
 'pcm_fftMag_mfcc_sma[1]_stddev',
 'pcm_fftMag_mfcc_sma[1]_skewness',
 'pcm_fftMag_mfcc_sma[1]_kurtosis',
 'pcm_fftMag_mfcc_sma[2]_max',
 'pcm_fftMag_mfcc_sma[2]_min',
 'pcm_fftMag_mfcc_sma[2]_range',
 'pcm_fftMag_mfcc_sma[2]_maxPos',
 'pcm_fftMag_mfcc_sma[2]_minPos',
 'pcm_fftMag_mfcc_sma[2

In [45]:
# emotion list to organize speech files by emotion 
emotions = ['anxiety', 'boredom', 'cold-anger', 'contempt',
           'despair', 'disgust', 'elation', 'happy','hot-anger',
           'interest', 'neutral', 'panic', 'pride', 'sadness',
           'shame']

speakers = ['cc', 'cl', 'gg', 'jg', 'mf', 'mk','mm']

def get_index(x): 
    return emotions.index(x)

def vectorize(x):
    return np.vectorize(get_index)(x)

In [52]:
from sklearn.preprocessing import StandardScaler
# classifies utterances using on-speaker-out cross validation
# returns averaged precision, recall, f1 
def clasify_utterances(df, emotions, norm=False):
    results = []
    # one-speaker-out cross validation
    for s in speakers:         
        train_df   = df.loc[df['speaker'] != s]
        train_data = np.array(train_df.loc[:,'pcm_RMSenergy_sma_max':'pcm_zcr_sma_de_kurtosis'])
#         print(train_data)
        y_train    = np.array(train_df['emotion'].values)
        y_train    = vectorize(y_train)

        test_df    = df.loc[df['speaker'] == s]
        test_data  = np.array(test_df.loc[:,'pcm_RMSenergy_sma_max':'pcm_zcr_sma_de_kurtosis'])
#         print(test_data)
        y_test     = np.array(test_df['emotion'].values)       
        y_test     = vectorize(y_test)

#         if norm: 
#             scaler = StandardScaler()
#             scaler.fit(train_data)
#             train_data = scaler.transform(train_data)
            
#             scaler.fit(test_data)
#             test_data = scaler.transform(test_data)
        
        
        rlf = RidgeClassifier(max_iter=1000).fit(train_data, y_train)
        y_pred = rlf.predict(test_data)
        
#         from sklearn.neural_network import MLPClassifier
#         mlp = MLPClassifier(hidden_layer_sizes=(30,30,30), max_iter=2000)

#         mlp.fit(train_data,y_train)
#         y_pred = mlp.predict(test_data)

    
#         clf = LinearSVC(random_state=0, tol=1e-5, max_iter=2000)
#         clf.fit(train_data, y_train)
#         y_pred = clf.predict(test_data)
    
        # get results
        precision = precision_score(y_test, y_pred, average="macro")
        recall = recall_score(y_test, y_pred, average="macro") 
        f1 = f1_score(y_test, y_pred, average="macro")
        
        print("\n")
        print("SPEAKER", s)
        print("TOTAL UTTERANCES", len(y_test))
        print(classification_report(y_test,y_pred,target_names=emotions))
        print("\n")
        
        results.append([precision, recall, f1])
        
    results = np.array(results)
    avg_results = np.mean(results, axis=0)
    
    # return average results 
    # precision, recall, f1 
    return avg_results 

print ("REGULAR")
# original data
res = clasify_utterances(df, emotions)

REGULAR


SPEAKER cc
TOTAL UTTERANCES 265
              precision    recall  f1-score   support

     anxiety       0.15      0.50      0.23        10
     boredom       0.05      0.07      0.06        15
  cold-anger       0.33      0.07      0.11        15
    contempt       0.22      0.50      0.30        22
     despair       0.25      0.33      0.29         9
     disgust       0.25      0.06      0.10        31
     elation       0.12      0.12      0.12        16
       happy       0.19      0.17      0.18        23
   hot-anger       0.47      0.64      0.55        14
    interest       0.50      0.35      0.41        17
     neutral       0.64      0.50      0.56        18
       panic       0.27      0.39      0.32        18
       pride       0.00      0.00      0.00        23
     sadness       0.00      0.00      0.00        13
       shame       0.12      0.10      0.11        21

   micro avg       0.23      0.23      0.23       265
   macro avg       0.24      0.25     

In [53]:
print ("NORM")
# normalized 
res_norm = clasify_utterances(norm_df, emotions, True)




NORM


SPEAKER cc
TOTAL UTTERANCES 265
              precision    recall  f1-score   support

     anxiety       0.15      0.50      0.23        10
     boredom       0.05      0.07      0.06        15
  cold-anger       0.33      0.07      0.11        15
    contempt       0.22      0.50      0.30        22
     despair       0.25      0.33      0.29         9
     disgust       0.25      0.06      0.10        31
     elation       0.12      0.12      0.12        16
       happy       0.19      0.17      0.18        23
   hot-anger       0.47      0.64      0.55        14
    interest       0.50      0.35      0.41        17
     neutral       0.64      0.50      0.56        18
       panic       0.27      0.39      0.32        18
       pride       0.00      0.00      0.00        23
     sadness       0.00      0.00      0.00        13
       shame       0.12      0.10      0.11        21

   micro avg       0.23      0.23      0.23       265
   macro avg       0.24      0.25      0.

In [54]:
print ("Results")
print(res)
print ("\nResults (Normalized data)")
print(res_norm)



Results
[0.24004332 0.2323745  0.20552424]

Results (Normalized data)
[0.24004332 0.2323745  0.20552424]
